# Python Functools

## Aim

- Give a digest of the module docs.
- Explain any jargon I had to look up.

> Link to this presentation:
> https://github.com/wxtim/Notes/blob/master/Functools%20Presentation.ipynb
>
> or
>
> (https://bit.ly/3CY0hu5)

### What is functools for?

> functions that act on or return other functions



Links between items in module are a bit tenous, but I have pulled out the following themes:

| Topic | Number of functions |
| --- | --- |
| Caching | 2.5 |
| Partial Functions | 2 |
| Type handling | 2 |
| Python 2 Upgrading | 1 |
| Other | 2 |

In [1]:
import functools

## Caching

![Squirrel Caching a nut?](https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/White-tailed_Antelope_Squirrel_DSC4931aavv.jpg/640px-White-tailed_Antelope_Squirrel_DSC4931aavv.jpg)

To store: Usually data in memory:

Also know as _memoization_.

- [Search on GH for 'from functools import LRU cache' ~62 000 hits](https://github.com/search?q=%22from+functools+import+lru_cache%22&type=code)
- [Search on GH for 'from functools import LRU cache' ~800 hits](https://github.com/search?q=%22from+functools+import+lru_cache%22&type=code)
- [Search on GH for 'from functools import cached_property' 3 100 hits](https://github.com/search?q=%22from+functools+import+cached_property%22&type=code)

In [14]:
# Simple un-cached function - It's recursive, so horrid!

def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

In [7]:
%%timeit
fibonacci(12)

59.5 µs ± 3.43 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
MY_CACHE = {}

def fibonacci(n):
    # Check for answer in MY_CACHE
    if n in MY_CACHE:
        return MY_CACHE[n]

    # Do calculation
    if n < 2:
        return n
    answer = fibonacci(n-1) + fibonacci(n-2)
    
    # Add to cache
    MY_CACHE[n] = answer
    return answer

In [9]:
%%timeit
fibonacci(12)

130 ns ± 1.16 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Functools has two different method decorators to save use this:
- functools.cache
- functools.lru_cache

In [10]:
@functools.cache
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

In [11]:
%%timeit
fibonacci(12)

96.2 ns ± 10.5 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [12]:
@functools.lru_cache
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

In [13]:
%%timeit
fibonacci(12)

88.4 ns ± 0.744 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


### Why are there two different caching decorators?


Turns out there aren't!
In the [source code](https://github.com/python/cpython/blob/bc4cde40339dd372960f27401d8fdaa4dab0f469/Lib/functools.py#L647):

```python

def cache(user_function, /):
    'Simple lightweight unbounded cache.  Sometimes called "memoize".'
    return lru_cache(maxsize=None)(user_function)
```

### So what does the maxsize option do?

If you set a `maxsize` the cache has to get rid of data when it runs out of space.


### Is it desirable to set maxsize?

It depends. 

- lru_cache has to check on the memory usage of the cache => cache should be faster

- cache could use arbitrarily large amounts of memory.

### What's an LRU cache?

LRU = Least Recently Used

Describes items dropped from cache when memory limit reached.

There are other caching options one could implement - they all describe which item in the cache to drop if you run out of space:
- First in first out (FIFO)
- First in last out (FILO)
- Least Frequently Used (LFU)
- Most Recently used (LRU)

### Real world example

From [`isodatetime` source](https://github.com/metomi/isodatetime/blob/99d509813bcee8c9fd64523221ef47d45a0c4a72/metomi/isodatetime/data.py#L2092)

In [19]:
LEAP_YEAR_FACTOR_TRUTHS = [(4, True), (100, False), (400, True)]

@functools.lru_cache(maxsize=100000)
def get_is_leap_year(year):
    """Return if year is a leap year."""
    year_is_leap = False
    for factor, is_leap_factor in LEAP_YEAR_FACTOR_TRUTHS:
        if year % factor == 0:
            year_is_leap = is_leap_factor
    return year_is_leap


### Glossed over in this section

`functools.cached_property`

## `functools.singledispatch`

### What is a "single dispatch generic function"?

[The docs say](https://docs.python.org/3/glossary.html#term-generic-function)

> A function composed of multiple functions implementing the same operation for different types


![A post box](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Post_box_PO21_15_%2813977727556%29.jpg/180px-Post_box_PO21_15_%2813977727556%29.jpg)

[Search on GH for 'from functools import singledispatch' ~7 300 hits](https://github.com/search?q=%22from+functools+import+singledispatch%22&type=code).


### Example

Create the dispatch function:

In [35]:
@functools.singledispatch
def my_function(arg):
    ...

register overloaded implementations:

In [42]:
@my_function.register
def _(arg: list):
    return ', '.join(arg)

@my_function.register
def _(arg: str):
    return ', '.join(arg.split(' '))

print(my_function(['My', 'Name', 'is', 'Tim']))

print(my_function('Je voudrais un verre de biére'))

My, Name, is, Tim
Je, voudrais, un, verre, de, biére


### Glossed over in this section

`singledispatchmethod` 

## `functools.total_ordering`

[Search on GH for 'from functools import total_ordering' ~27 000 hits](https://github.com/search?q=%22from+functools+import+total_ordering%22&type=code).


![Mammoth Comparison](https://upload.wikimedia.org/wikipedia/commons/e/e9/Mammoth_Size_comparison.png)


 `__eq__` + one of `__lt__`, `__le__`, `__gt__`, `__ge__`
 
 is enough information to create
 
 - all the others
 - `__ne__`

In [31]:
@functools.total_ordering
class Rectangle:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __eq__(self, other):
        return self.x * self.y == other.x * other.y
    
    def __gt__(self, other):
        return self.x * self.y > other.x * other.y

In [32]:
green = Rectangle(2, 5)
blue = Rectangle(5, 2)

print(blue < green)
print(blue <= green)

False
True


| Good | Bad |
| --- | --- |
| ensures consistency | may be slower |
|saves typing| may make traceback harder to read| 

## Reduce


![Jam in a pan](https://upload.wikimedia.org/wikipedia/commons/thumb/6/63/-2021-09-14_Saucepan_of_boiling_Homemade_Blackberry_%26_Apple_jelly%2C_Trimingham%2C_Norfolk.JPG/320px--2021-09-14_Saucepan_of_boiling_Homemade_Blackberry_%26_Apple_jelly%2C_Trimingham%2C_Norfolk.JPG)

[Search on GH for 'from functools import reduce' 225 000 hits](https://github.com/search?q=%22from+functools+import+reduce%22&type=code)

Collapse an interable down to a single thing according to a function:

In [53]:
# Sum of the first 3 numbers:
functools.reduce(lambda x, y: x+y, [1, 2, 3])

6

In [55]:
# Sum of the first 6 fibonacci numbers:
functools.reduce(lambda x, y: x+y, [fibonacci(i) for i in range(7)])

20

## Partial

[380 000 imports of `from functools import partial` on GH](https://github.com/search?q=%22from+functools+import+partial%22&type=Code)

Create a function from another function, hard-setting some args and kwargs in the process:

### Simple examples

In [59]:
from sys import stderr
printerr = functools.partial(print, file=stderr)
printerr('This function will print to stdout')

This function will print to stdout


In [76]:
def is_odd(n):
    if n % 2 == 0:
        return False
    return True


filterodd = functools.partial(filter, is_odd)
[i for i in filterodd([1,2,3,4])]

[1, 3]

### Glossed over in this section:

`partialmethod` - partial for class methods.

## Glossed over

`cmp_to_key` - Designed to convert Python 2 comparison functions to key functions. Handly if you still have lots of Python 2 hanging around.

## Summary

There's a bunch of somewhat tenuously connected things in functools, but they are potentially useful and well used.

- Create Partial objects i.e. Functions with some args/kwargs filled in.
- Reduce an interator by some method.
- Caching
- Rich Comparisons
- Single dispatch
